In [0]:
dbutils.widgets.text("EXTENDED_SALES_TABLE_PATH", "")
dbutils.widgets.text("UC_CATALOG_NAME", "")
dbutils.widgets.text("UC_SCHEMA_NAME", "")

In [0]:
EXTENDED_SALES_TABLE_PATH = dbutils.widgets.get("EXTENDED_SALES_TABLE_PATH")
if not EXTENDED_SALES_TABLE_PATH :
    dbutils.notebook.exit("Please provide a path for the ExtendedSalesDeltaTable.")

UC_CATALOG_NAME = dbutils.widgets.get("UC_CATALOG_NAME")
if not UC_CATALOG_NAME:
    dbutils.notebook.exit("Please provide catalog name.")

UC_SCHEMA_NAME = dbutils.widgets.get("UC_SCHEMA_NAME")
if not UC_SCHEMA_NAME:
    dbutils.notebook.exit("Please provide schema name.")

In [0]:
from delta.tables import DeltaTable

if DeltaTable.isDeltaTable(spark, EXTENDED_SALES_TABLE_PATH):
    dbutils.notebook.exit(f"ExtendedSalesDeltaTable has already been set up (path: {EXTENDED_SALES_TABLE_PATH}). If you need to configure specific components, please do so manually.")

In [0]:
spark.sql(f"""
CREATE SCHEMA IF NOT EXISTS {UC_CATALOG_NAME}.{UC_SCHEMA_NAME}      
""")

In [0]:
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {UC_CATALOG_NAME}.{UC_SCHEMA_NAME}.ExtendedSalesDeltaTable 
(
  SaleID LONG NOT NULL,
  SaleDate DATE NOT NULL,
  Region STRING NOT NULL
)
USING DELTA
LOCATION '{EXTENDED_SALES_TABLE_PATH}';
""")

In [0]:
spark.sql(f"""
ALTER TABLE {UC_CATALOG_NAME}.{UC_SCHEMA_NAME}.ExtendedSalesDeltaTable 
ADD CONSTRAINT chk_region_valid CHECK (Region IN ('north', 'south', 'west'))
""")

In [0]:
spark.sql(f"""
ALTER TABLE {UC_CATALOG_NAME}.{UC_SCHEMA_NAME}.ExtendedSalesDeltaTable
SET TBLPROPERTIES (
  'delta.autoOptimize.optimizeWrite' = 'true',
  'delta.autoOptimize.autoCompact' = 'true'
);
""")

In [0]:
dbutils.notebook.exit("ExtendedSalesDeltaTable setup successful.")
